In [1]:
import requests
from bs4 import BeautifulSoup

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [15]:
r = requests.get('https://www.msss.com.my/mjss/')

In [16]:
soup = BeautifulSoup(r.content)
a = soup.find_all('a', {'class': 'list-group-item'})
pages = [a_.get('href') for a_ in a if a_.get('href').startswith('v')]
pages

['v27.php',
 'v26.php',
 'v25.php',
 'v24.php',
 'v23.php',
 'v22.php',
 'v21.php',
 'v20.php',
 'v19.php',
 'v18.php',
 'v17.php',
 'v16.php',
 'v15.php',
 'v14.php',
 'v13.php',
 'v12.php',
 'v11.php',
 'v10.php',
 'v09.php',
 'v08.php',
 'v07.php',
 'v06.php',
 'v05.php',
 'v04.php',
 'v03.php',
 'v02.php',
 'v01.php']

In [17]:
from tqdm import tqdm

urls = []
for p in tqdm(pages):
    p = f'https://www.msss.com.my/mjss/{p}'
    r = requests.get(p)
    soup = BeautifulSoup(r.content)
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href').endswith('.pdf')]
    urls.extend(a)
    
urls

100%|███████████████████████████████████████████| 27/27 [00:05<00:00,  5.37it/s]


['../../mjss/Full Text/vol27/V27_01.pdf',
 '../../mjss/Full Text/vol27/V27_02.pdf',
 '../../mjss/Full Text/vol27/V27_03.pdf',
 '../../mjss/Full Text/vol27/V27_04.pdf',
 '../../mjss/Full Text/vol27/V27_05.pdf',
 '../../mjss/Full Text/vol27/V27_06.pdf',
 '../../mjss/Full Text/vol27/V27_07.pdf',
 '../../mjss/Full Text/vol27/V27_08.pdf',
 '../../mjss/Full Text/vol27/V27_09.pdf',
 '../../mjss/Full Text/vol27/V27_10.pdf',
 '../../mjss/Full Text/vol27/V27_11.pdf',
 '../../mjss/Full Text/vol27/V27_12.pdf',
 '../../mjss/Full Text/vol27/V27_13.pdf',
 '../../mjss/Full Text/vol27/V27_14.pdf',
 '../../mjss/Full Text/vol27/V27_15.pdf',
 '../../mjss/Full Text/vol27/V27_16.pdf',
 '../../mjss/Full Text/vol27/V27_17.pdf',
 '../../mjss/Full Text/vol27/V27_18.pdf',
 '../../mjss/Full Text/vol26/V26_01.pdf',
 '../../mjss/Full Text/vol26/V26_02.pdf',
 '../../mjss/Full Text/vol26/V26_03.pdf',
 '../../mjss/Full Text/vol26/V26_04.pdf',
 '../../mjss/Full Text/vol26/V26_05.pdf',
 '../../mjss/Full Text/vol26/V26_0

In [18]:
urls = sorted(list(set(urls)))
len(urls)

234

In [19]:
!mkdir msss.com.my

In [24]:
import os

pdfs = urls
for i in tqdm(range(len(pdfs))):
    filename = os.path.join('msss.com.my', f'{i}.pdf')
    if os.path.exists(filename):
        continue
        
    u = pdfs[i].split('/mjss/')[1]
    u = f'https://www.msss.com.my/mjss/{u}'
        
    r = requests.get(u)
    with open(filename, 'wb') as fopen:
        fopen.write(r.content)

100%|█████████████████████████████████████████| 234/234 [15:58<00:00,  4.09s/it]


In [27]:
from glob import glob
from tika import parser
import json

TIKA_HOST = 'http://localhost:9998'

In [29]:
files = glob('msss.com.my/*.pdf')

with open('msss.com.my.jsonl', 'w') as fopen:
    for f in tqdm(files):
        try:
            raw_xml = parser.from_file(f, TIKA_HOST, xmlContent=True)
            body = BeautifulSoup(raw_xml['content']).find('body')
            t = '\n'.join(body.findAll(text=True))
            fopen.write(f'{json.dumps(t)}\n')
        except:
            pass

100%|█████████████████████████████████████████| 234/234 [00:11<00:00, 21.27it/s]
